In [2]:
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import cookielib
import urllib2
import requests
import re
import json
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.orm.session import sessionmaker

In [67]:
#url = 'http://foxnews.com/health.html'
url = 'http://foxnews.com/tech.html'

In [68]:
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [69]:
results = soup.find_all('article')

In [62]:
links

[u'/category/world/global-economy.html',
 u'/category/world/conflicts/iran.html',
 u'/category/world/world-regions/asia.html',
 u'/category/world/world-regions/middle-east.html',
 u'/category/lifestyle.html',
 u'/world/regions.html',
 u'/world/regions.html',
 u'/world/regions.html',
 u'/world/regions.html',
 u'/world/regions.html',
 u'/world/regions.html',
 u'/opinion/2017/02/04/congos-violent-kleptocracy-at-crossroads.html',
 u'/opinion/2017/02/03/give-trump-three-cheers-for-standing-up-to-anti-american-deal-with-australia.html',
 u'/opinion/2017/02/03/very-real-very-dangerous-threat-iranian-regime.html',
 u'/opinion/2017/02/02/senators-cornyn-rubio-young-its-time-for-action-against-iran.html']

In [70]:
links = []
authors = []
dates = []
bodies = []
titles = []
sections = []

for x in results:
    link = x.find('a')['href']
    match1 = re.search('^http://', link)
    match2 = re.search('/slideshow', link)
    if not match1 and not match2:
        links.append(link)

for link in links:
    url = 'http://www.foxnews.com' + link
    response = requests.get(url)
    html = response.content
    new_soup = BeautifulSoup(html, 'html.parser')
    
    # get title
    title = new_soup.find('title').text.strip().replace('| Fox News','')
    titles.append(title)
    
    # get body
    all_p = ''
    body = new_soup.find('div', attrs = {'class': 'article-text'})
    paragraphs = body.find_all('p')
    for p in paragraphs:
        p = p.text.strip().encode('ascii','ignore')
        all_p = all_p + p
    bodies.append(all_p)
    
    # get date
    date = new_soup.find('meta', attrs = {'name': 'dc.date'})['content']
    dates.append(date)
    
    # get section
    section = new_soup.find('meta', attrs = {'name': "prism.section"})['content']
    sections.append(section)
    
    # get author
    author_div = new_soup.find('div', attrs = {'class': 'article-info'})
    try:
        author = author_div.find('p').text.strip()
    except AttributeError:
        author = "None found"
    authors.append(author)
        

In [71]:
my_dict = {
        'title': titles, 'link': links, 'author': authors, 'body': bodies,
        'section': sections, 'date': dates
    }

In [72]:
my_df = pd.DataFrame(my_dict)

In [73]:
my_df.head()

,author,body,date,link,section,title
0,None found,Some 300 drones lit up the sky in a spectacula...,2017-02-06,/tech/2017/02/06/how-drones-lit-up-sky-in-lady...,tech,How drones lit up the sky in Lady Gaga's Super...
1,By \t\n\t\t \n\t\t ...,It takes a lot of courage to make changes that...,2017-02-07,/tech/2017/02/07/apple-didnt-just-fix-its-dong...,tech,Apple didn't just fix its dongle hell problem ...
2,By \t\n\t\t \n\t\t ...,It's been a hectic start for President Donald ...,2017-02-07,/tech/2017/02/07/tesla-and-spacex-join-legal-f...,tech,Tesla and SpaceX join legal fight against Trum...
3,None found,If you havent gotten around to using a passwor...,2017-02-07,/tech/2017/02/07/everything-need-to-know-about...,tech,Everything You Need to Know About Password Man...
4,By \t\n\t\t \n\t\t ...,"Riding on a wave of nostalgia, the NES Classic...",2017-02-06,/tech/2017/02/06/nintendo-has-already-sold-1-5...,tech,Nintendo has already sold 1.5 million NES Clas...


In [74]:
engine = create_engine('postgresql://teresaborcuch@localhost:5433/capstone')

In [75]:
# clear staging
clear_staging_query = 'DELETE FROM fox_staging *;'

In [76]:
# create sqlalchemy session and clear staging
Session = sessionmaker(bind=engine)
session = Session()

engine.execute(clear_staging_query)
session.commit()

In [77]:
# add dataframe to staging
my_df.to_sql('fox_staging', engine, if_exists = 'append', index = False)

In [78]:
move_unique_query = '''
INSERT INTO fox_news (title, date, author, body, link, section)
SELECT title, date, author, body, link, section
FROM fox_staging
WHERE NOT EXISTS (SELECT title, date, author, body, link, section
FROM fox_news
WHERE fox_news.title = fox_staging.title);
'''

In [79]:
# run the move_unique_query
engine.execute(move_unique_query)
session.commit()
session.close()